In [1]:
import pandas as pd
import numpy as np

#### What data is available?
For this competition we are given both a train and test dataset, which are synthetically generated from an existing dataset. If these datasets are combined this could give far more data which may result in higher auc roc scores in the competition.

Lets have a quick look at the provided train dataset

In [2]:
train_df = pd.read_csv('../data/train.csv')
train_columns = train_df.columns.to_list()
train_df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


lets have a quick look at the original data this was generated using

In [3]:
og_data = pd.read_csv('../data/credit_risk_dataset.csv')
og_columns = og_data.columns.to_list()
og_data.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [4]:
columns_only_in_train = list(set(train_columns) - set(og_columns))
columns_only_in_og = list(set(og_columns) - set(train_columns))
shared_columns = list(set(train_columns).intersection(og_columns))

print('Columns only in train:', columns_only_in_train)
print('Columns only in og:', columns_only_in_og)
print('Shared columns:', shared_columns)


Columns only in train: ['id']
Columns only in og: []
Shared columns: ['person_home_ownership', 'loan_int_rate', 'person_age', 'cb_person_default_on_file', 'person_emp_length', 'loan_status', 'loan_amnt', 'cb_person_cred_hist_length', 'loan_intent', 'loan_grade', 'loan_percent_income', 'person_income']


We can clearly see that all the columns line up apart from the 'id' which is not a feature that is used to predict the target. This is great and means that the data may be easy to integrate into a larger train set as long as the values are the same format.

Lets check that now

In [5]:
def check_columns(df1, df2,shared_columns):
    format_check_results = {}

    for col in shared_columns:
        df1_type = df1[col].dtype
        df2_type = df2[col].dtype
        
        if df1_type != df2_type:
            format_check_results[col] = {
                'format_match': False,
                'df1': df1_type,
                'df2': df2_type
            }
        else:
            if pd.api.types.is_numeric_dtype(df1[col]) and pd.api.types.is_numeric_dtype(df2[col]):
                format_check_results[col] = {
                    'format_match': True,
                    'dtype': df1_type,
                    'df1_values_range': (df1[col].min(), df1[col].max()),
                    'df2_values_range': (df2[col].min(), df2[col].max())
                }
            
            elif pd.api.types.is_string_dtype(df1[col]) or pd.api.types.is_categorical_dtype(df2[col]):
                df1_unique = set(df1[col].dropna().unique())
                df2_unique = set(df2[col].dropna().unique())
                
                format_check_results[col] = {
                    'format_match': df1_unique == df2_unique,
                    'df1_unique_values': df1_unique,
                    'df2_unique_values': df2_unique
                }
            
            else:
                format_check_results[col] = {
                    'format_match': True,
                    'dtype': df1_type
                }

    format_check_df = pd.DataFrame.from_dict(format_check_results, orient='index')

    format_check_df.reset_index(inplace=True)
    format_check_df.rename(columns={'index': 'Column Name'}, inplace=True)
    return format_check_df   

check_columns(train_df, og_data, shared_columns)

,Column Name,format_match,df1_unique_values,df2_unique_values,dtype,df1_values_range,df2_values_range
0,person_home_ownership,True,"{RENT, OTHER, MORTGAGE, OWN}","{RENT, OTHER, MORTGAGE, OWN}",NaN,NaN,NaN
1,loan_int_rate,True,NaN,NaN,float64,"(5.42, 23.22)","(5.42, 23.22)"
2,person_age,True,NaN,NaN,int64,"(20, 123)","(20, 144)"
3,cb_person_default_on_file,True,"{Y, N}","{Y, N}",NaN,NaN,NaN
4,person_emp_length,True,NaN,NaN,float64,"(0.0, 123.0)","(0.0, 123.0)"
5,loan_status,True,NaN,NaN,int64,"(0, 1)","(0, 1)"
6,loan_amnt,True,NaN,NaN,int64,"(500, 35000)","(500, 35000)"
7,cb_person_cred_hist_length,True,NaN,NaN,int64,"(2, 30)","(2, 30)"
8,loan_intent,True,"{DEBTCONSOLIDATION, PERSONAL, HOMEIMPROVEMENT,...","{DEBTCONSOLIDATION, PERSONAL, HOMEIMPROVEMENT,...",NaN,NaN,NaN
9,loan_grade,True,"{E, G, A, B, D, C, F}","{E, G, A, B, D, C, F}",NaN,NaN,NaN


we can see that the loan status percentages are similar so the datasets have the same bias in loan status approval rate.

In [6]:
print('og loan_status_value_counts', og_data['loan_status'].value_counts())
print('og percentage of loan_status_value_counts', og_data['loan_status'].value_counts(normalize=True))
print('train loan_status_value_counts', train_df['loan_status'].value_counts())
print('train percentage of loan_status_value_counts', train_df['loan_status'].value_counts(normalize=True))

og loan_status_value_counts loan_status
0    25473
1     7108
Name: count, dtype: int64
og percentage of loan_status_value_counts loan_status
0    0.781836
1    0.218164
Name: proportion, dtype: float64
train loan_status_value_counts loan_status
0    50295
1     8350
Name: count, dtype: int64
train percentage of loan_status_value_counts loan_status
0    0.857618
1    0.142382
Name: proportion, dtype: float64


We can see that the format is also the same for all columns, now lets join them together to gain a much larger dataset, I did this using sql for a bit of practice but it could also be done with pandas, for the sql method in bash run
``` 
cd data
sqlite3 loan_approval.db < database_setup.sql
```
To check the resulting csv lets examine it here


In [7]:
merged_df = pd.read_csv('../data/merged_dataset.csv')
merged_df.head()


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [8]:
check_columns(merged_df, train_df,shared_columns)

,Column Name,format_match,df1_unique_values,df2_unique_values,dtype,df1_values_range,df2_values_range
0,person_home_ownership,True,"{RENT, OTHER, MORTGAGE, OWN}","{RENT, OTHER, MORTGAGE, OWN}",NaN,NaN,NaN
1,loan_int_rate,True,NaN,NaN,float64,"(5.42, 23.22)","(5.42, 23.22)"
2,person_age,True,NaN,NaN,int64,"(20, 144)","(20, 123)"
3,cb_person_default_on_file,True,"{Y, N}","{Y, N}",NaN,NaN,NaN
4,person_emp_length,True,NaN,NaN,float64,"(0.0, 123.0)","(0.0, 123.0)"
5,loan_status,True,NaN,NaN,int64,"(0, 1)","(0, 1)"
6,loan_amnt,True,NaN,NaN,int64,"(500, 35000)","(500, 35000)"
7,cb_person_cred_hist_length,True,NaN,NaN,int64,"(2, 30)","(2, 30)"
8,loan_intent,True,"{DEBTCONSOLIDATION, PERSONAL, HOMEIMPROVEMENT,...","{DEBTCONSOLIDATION, PERSONAL, HOMEIMPROVEMENT,...",NaN,NaN,NaN
9,loan_grade,True,"{E, G, A, B, D, C, F}","{E, G, A, B, D, C, F}",NaN,NaN,NaN
